In [ ]:
import pandas as pd
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.utils import resample
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from nltk.corpus import stopwords

tr = open("train.txt").read().split('\n')
val = open("val.txt").read().split('\n')
ts = open("test.txt").read().split('\n')

stopWords = set(stopwords.words('english'))
# Remove these words from the stopwords list that may be important to classifying emotion
stopWords = stopWords - {'not', 'dont', 'doesnt', 'didnt', 'no', 'down', 'very', 'under', 'below', 'off', 'on', 'up'}

train = [d.split(';') for d in tr]
train = train[:-1]

for s in train:
    if " t " in s[0]:
        s[0] = s[0].replace(" t ", "t ")

validation = [d.split(';') for d in val]
validation = validation[:-1]

for s in validation:
    if " t " in s[0]:
        s[0] = s[0].replace(" t ", "t ")

test = [d.split(';') for d in ts]
test = test[:-1]

for s in test:
    if " t " in s[0]:
        s[0] = s[0].replace(" t ", "t ")

In [ ]:
train_df = pd.DataFrame(data=train + validation, columns=['doc', 'class'])
test_df = pd.DataFrame(data=test, columns=['doc','class'])

train_df['class'].value_counts()

joy         6066
sadness     5216
anger       2434
fear        2149
love        1482
surprise     653
Name: class, dtype: int64

In [ ]:

anger = train_df[train_df['class'] == 'anger']
fear = train_df[train_df['class'] == 'fear']
sadness = train_df[train_df['class'] == 'sadness']
joy = train_df[train_df['class'] == 'joy']
love = train_df[train_df['class'] == 'love']

#sadness = resample(sadness, replace=True, n_samples=5000)
anger = resample(anger, replace=True, n_samples=3500)
fear = resample(fear, replace=True, n_samples=3500)
love = resample(love, replace=True, n_samples=6000)
#joy = resample(joy, replace=True, n_samples=6000)

train_df = pd.concat([joy, love, anger, fear, sadness])

print(train_df.shape)
train_df['class'].value_counts()

(24282, 2)


joy        6066
love       6000
sadness    5216
fear       3500
anger      3500
Name: class, dtype: int64

In [ ]:
test_df = test_df[test_df['class'] != 'surprise']
test_df['class'].value_counts()

joy        695
sadness    581
anger      275
fear       224
love       159
Name: class, dtype: int64

In [ ]:
train_df['class'].replace(['sadness', 'anger', 'fear'], 'unpleasant', inplace=True)
train_df['class'].replace(['love', 'joy'], 'pleasant', inplace=True)

test_df['class'].replace(['sadness', 'anger', 'fear'], 'unpleasant', inplace=True)
test_df['class'].replace(['love', 'joy'], 'pleasant', inplace=True)

In [ ]:
test_classes = test_df['class'].values.tolist()

In [ ]:
vectorizer = TfidfVectorizer(analyzer='word', stop_words=stopWords)
tfidf_train = vectorizer.fit_transform(train_df['doc'].values.tolist())

# Initiate KNN classifier
#knn_classifier = KNeighborsClassifier(n_neighbors=18, weights='distance', p=2, n_jobs=-1)
knn_classifier = KNeighborsClassifier()

params = {'n_neighbors':list(range(1,30)),
          'weights':['uniform', 'distance'],
          'p': [1,2],
          'algorithm':['auto'],
          'n_jobs':[-1]}
knn_classifier = GridSearchCV(knn_classifier, param_grid=params, n_jobs=-1)

knn_classifier.fit(X=tfidf_train, y=train_df['class'])

print("\nBest KNN params: ", knn_classifier.best_params_)

# KNN on test data
# TFIDF on test data
tfidf_test = vectorizer.transform(test_df['doc'].values.tolist())
test_pred_knn = knn_classifier.predict(tfidf_test)

NameError: ignored

In [ ]:

print("\n---KNN Results---\n")

print(classification_report(y_true=test_classes, y_pred=test_pred_knn))
print("test confusion matrix\n", confusion_matrix(y_true=test_classes, y_pred=test_pred_knn))
plot_confusion_matrix(estimator=knn_classifier, X=tfidf_test, y_true=test_classes)


---KNN Results---



NameError: ignored